# Otimização do modelo

## Importar bibliotecas

In [ ]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import pandas as pd
import numpy as np

import random
from itertools import product

import pickle

In [2]:
subsistema = 'S'

## Importando dados

In [3]:
dados_treinamento = pd.read_csv(f'../../../Preprocessamento/{subsistema}_treinamento.csv', index_col='DataHora')
dados_treinamento.index = pd.to_datetime(dados_treinamento.index, format="%Y-%m-%d %H:%M:%S")
dados_treinamento

,Carga,Temperatura,seg,ter,qua,qui,sex,sab,dom,seno_dia_semana,...,mai,jun,jul,ago,set,out,nov,dez,seno_mes,cosseno_mes
DataHora,,,,,,,,,,,,,,,,,,,,,
2018-01-03 00:00:00,-0.382235,0.028019,0,0,1,0,0,0,0,0.974928,...,0,0,0,0,0,0,0,0,5.000000e-01,0.866025
2018-01-03 01:00:00,-0.482857,0.054438,0,0,1,0,0,0,0,0.974928,...,0,0,0,0,0,0,0,0,5.000000e-01,0.866025
2018-01-03 02:00:00,-0.551445,0.012211,0,0,1,0,0,0,0,0.974928,...,0,0,0,0,0,0,0,0,5.000000e-01,0.866025
2018-01-03 03:00:00,-0.595054,-0.014208,0,0,1,0,0,0,0,0.974928,...,0,0,0,0,0,0,0,0,5.000000e-01,0.866025
2018-01-03 04:00:00,-0.618897,-0.040627,0,0,1,0,0,0,0,0.974928,...,0,0,0,0,0,0,0,0,5.000000e-01,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 19:00:00,0.016627,-0.008793,0,0,0,0,0,0,1,-0.781831,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.000000
2023-12-31 20:00:00,0.070279,-0.061631,0,0,0,0,0,0,1,-0.781831,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.000000
2023-12-31 21:00:00,0.140780,-0.061631,0,0,0,0,0,0,1,-0.781831,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.000000


In [4]:
dados_treinamento_2019 = dados_treinamento[dados_treinamento.index.year != 2018]
dados_treinamento_2020 = dados_treinamento_2019[dados_treinamento_2019.index.year != 2019]
dados_treinamento_2021 = dados_treinamento_2020[dados_treinamento_2020.index.year != 2020]
dados_treinamento_2022 = dados_treinamento_2021[dados_treinamento_2021.index.year != 2021]
dados_treinamento_2023 = dados_treinamento_2022[dados_treinamento_2022.index.year != 2022]

In [5]:
dados_validacao = pd.read_csv(f'../../../Preprocessamento/{subsistema}_validacao.csv', index_col='DataHora')
dados_validacao.index = pd.to_datetime(dados_validacao.index, format="%Y-%m-%d %H:%M:%S")
dados_validacao

,Carga,Temperatura,seg,ter,qua,qui,sex,sab,dom,seno_dia_semana,...,mai,jun,jul,ago,set,out,nov,dez,seno_mes,cosseno_mes
DataHora,,,,,,,,,,,,,,,,,,,,,
2023-12-30 00:00:00,-0.170209,0.056298,0,0,0,0,0,1,0,-0.974928,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.0
2023-12-30 01:00:00,-0.265651,0.040490,0,0,0,0,0,1,0,-0.974928,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.0
2023-12-30 02:00:00,-0.331625,-0.001737,0,0,0,0,0,1,0,-0.974928,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.0
2023-12-30 03:00:00,-0.388789,-0.014208,0,0,0,0,0,1,0,-0.974928,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.0
2023-12-30 04:00:00,-0.430474,-0.045823,0,0,0,0,0,1,0,-0.974928,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,0.070294,0.105844,0,1,0,0,0,0,0,0.781831,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.0
2024-12-31 20:00:00,0.111629,-0.002431,0,1,0,0,0,0,0,0.781831,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.0
2024-12-31 21:00:00,0.130015,-0.012999,0,1,0,0,0,0,0,0.781831,...,0,0,0,0,0,0,0,1,-2.449294e-16,1.0


## Criação dos Datasets

In [ ]:
class Dados(Dataset):
    def __init__(self, dados, modelo, entrada_carga = 48, horas_de_previsao = 24, horizonte = 2):
        # Recebendo e tratando os dados
        self.dados = dados

        # Pega dados de carga
        self.carga = self.dados['Carga']

        # Pega dados de temperatura
        self.temperatura = self.dados['Temperatura']
        
        # Pega dados de dia da semana no formato sen cos
        self.lista_dias_semana_S = list(self.dados.columns[2:4])
        self.dias_semana_S = self.dados[self.lista_dias_semana_S]

        # Pega dados de mes no formato sen cos
        self.lista_mes_S = list(self.dados.columns[4:])
        self.meses_S = self.dados[self.lista_mes_S]

        # Configurando outras variaveis
        self.entrada_carga = 48
        self.horas_de_previsao = horas_de_previsao
        self.horizonte = horizonte

        # Separa os dados
        self.entrada = []
        self.previsao = []
        for idx in range((len(self.carga) // 24) - self.entrada_carga // 24 - self.horizonte + 1):
            # Pula dia a dia
            idx = idx * 24
            
            # Salva o historico de carga
            carga_hist = self.carga[idx: idx + self.entrada_carga]

            # Salva os dados de temperatura do dia da previsão
            temperatura_H = self.temperatura[idx + self.entrada_carga + (self.horizonte - 1) * 24 : idx + self.entrada_carga + (self.horizonte) * 24]

            # Salva os dados de temperatura mínima, media e máxima
            temperatura_P = pd.Series([temperatura_H.min(), temperatura_H.mean(), temperatura_H.max()])
            
            # Salva os dados de dia da semana no formato sen cos
            dia_S = self.dias_semana_S.iloc[idx + self.entrada_carga + (self.horizonte - 1) * 24]

            # Salva os dados de mes no formato sen cos
            mes_S = self.meses_S.iloc[idx + self.entrada_carga + (self.horizonte - 1) * 24]

            # Gera as entradas concatenando os dados
            if modelo == 'M1_1':
                previsor_concat = pd.concat([carga_hist]).values
            elif modelo == 'M2_1':
                previsor_concat = pd.concat([carga_hist, temperatura_H]).values
            elif modelo == 'M2_2':
                previsor_concat = pd.concat([carga_hist, temperatura_P]).values
            elif modelo == 'M2_3':
                previsor_concat = pd.concat([carga_hist, dia_S]).values
            elif modelo == 'M2_4':
                previsor_concat = pd.concat([carga_hist, mes_S]).values
            elif modelo == 'M3_1':
                previsor_concat = pd.concat([carga_hist, temperatura_H, dia_S]).values
            elif modelo == 'M3_2':
                previsor_concat = pd.concat([carga_hist, temperatura_H, mes_S]).values
            elif modelo == 'M3_3':
                previsor_concat = pd.concat([carga_hist, temperatura_P, dia_S]).values
            elif modelo == 'M3_4':
                previsor_concat = pd.concat([carga_hist, temperatura_P, mes_S]).values
            elif modelo == 'M3_5':
                previsor_concat = pd.concat([carga_hist, dia_S, mes_S]).values
            elif modelo == 'M4_1':
                previsor_concat = pd.concat([carga_hist, temperatura_H, dia_S, mes_S]).values
            elif modelo == 'M4_2':
                previsor_concat = pd.concat([carga_hist, temperatura_P, dia_S, mes_S]).values
                
            self.entrada.append(previsor_concat)

            # Gera os dados de saida
            saida_carga = self.carga[idx + self.entrada_carga + (self.horizonte - 1) * 24 : idx + self.entrada_carga + (self.horizonte) * 24]
            self.previsao.append(saida_carga)
            
        self.entrada = torch.tensor(np.array(self.entrada), dtype = torch.float32)
        self.previsao = torch.tensor(np.array(self.previsao), dtype = torch.float32)

    def __len__(self):
        return (len(self.carga) // 24) - self.entrada_carga // 24 - self.horizonte + 1

    def __getitem__(self, idx):
        return self.entrada[idx], self.previsao[idx]

## Criando modelo LSTM e earlystop

In [7]:
class LSTM(nn.Module):
    def __init__(self, n_entrada, n_primeira_camada, n_segunda_camada, n_saida):
        super().__init__()
        self.lstm1 = nn.LSTM(
            input_size=n_entrada,
            hidden_size=n_primeira_camada,
            batch_first=True
        )
        self.lstm2 = nn.LSTM(
            input_size=n_primeira_camada,
            hidden_size=n_segunda_camada,
            batch_first=True
        )
        self.features = nn.Sequential(
            nn.Linear(n_segunda_camada, n_saida),
            nn.Tanh(),
        )

    def forward(self, X):
        if X.dim() == 2:
            X = X.unsqueeze(1)
        
        saida, _ = self.lstm1(X)
        saida, _ = self.lstm2(saida)
        saida = saida[:, -1, :]
        saida = self.features(saida)
        return saida

In [ ]:
class EarlyStopping:
    def __init__(self, patience=25, min_epoch = 100):
        """
        patience: quantas épocas esperar antes de parar se não houver melhora
        """
        self.patience = patience
        self.best_loss = np.inf
        self.counter = 0
        self.min_epoch = min_epoch
        self.contador_epocas = 0

    def __call__(self, val_loss):
        self.contador_epocas += 1
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience and self.contador_epocas >= self.min_epoch:
                return True  # Sinaliza que o treinamento deve parar
            return False

## Criando treinamento e validação

In [9]:
def treinamento(rede, treinamento_dataloader, validacao_dataloader, otimizador, criterio):  
    rede.train()
    perda_treino = []
    for entrada, saida in treinamento_dataloader:
        otimizador.zero_grad()
        previsao = rede(entrada)
        perda = criterio(previsao, saida)
        perda_treino.append(perda.item())
        perda.backward()
        otimizador.step()
        

    # Validação
    rede.eval()
    perda_validacao = []
    with torch.no_grad():
        for entrada, saida in validacao_dataloader:
            previsao = rede(entrada)
            perda_validacao.append(criterio(previsao, saida).item())
    
    # Cálculo da média das perdas
    perda_treino = np.asarray(perda_treino)
    perda_validacao = np.asarray(perda_validacao)

    return perda_treino.mean(), perda_validacao.mean()

## Função perda

In [ ]:
def treinar(params):
    # Definição dos hiperparâmetros que serão otimizados
    modelo_entrada, n_neuronios_1_lstm, n_neuronios_2_lstm, ano_entrada, taxa_aprendizado, otm  = params

    lista_modelo_entrada = (
        [f"M1_{i}" for i in range(1, 2)] +
        [f"M2_{i}" for i in range(1, 5)] +
        [f"M3_{i}" for i in range(1, 6)] + 
        [f"M4_{i}" for i in range(1, 3)]
    )
    lista_modelo_n_entradas = [48, 48+24, 48+3, 48+2, 48+2, 48+24+2, 48+24+2, 48+3+2, 48+3+2, 48+2+2, 48+24+2+2, 48+3+2+2]
    n_entradas = lista_modelo_n_entradas[modelo_entrada]
    modelo_entrada = lista_modelo_entrada[modelo_entrada]

    lista_n_neuronios = [24, round(0.25*(2*n_entradas-24)+24), round(0.5*(2*n_entradas-24)+24), round(0.75*(2*n_entradas-24)+24), 2*n_entradas]
    n_neuronios_1_lstm = lista_n_neuronios[n_neuronios_1_lstm]

    lista_n_neuronios = [24, round(0.25*(n_neuronios_1_lstm-24)+24), round(0.5*(n_neuronios_1_lstm-24)+24), round(0.75*(n_neuronios_1_lstm-24)+24), n_neuronios_1_lstm]
    n_neuronios_2_lstm = lista_n_neuronios[n_neuronios_2_lstm]

    lista_ano_entrada = [2018, 2020, 2022]
    ano_entrada = lista_ano_entrada[ano_entrada]

    lista_taxa_aprendizado = [0.01, 0.001, 0.0001]
    taxa_aprendizado = lista_taxa_aprendizado[taxa_aprendizado]

    lista_otm = ['Adam', 'Rprop', 'RMSprop']
    otm = lista_otm[otm]

    num_seeds = 10
    n_earlystop = 50
    epocas = 1000
    lista_melhor_loss = []

    for seed in range(1, num_seeds + 1):
        # Criando dados
        if ano_entrada == 2018:
            treinamento_dataset = Dados(dados_treinamento, modelo_entrada)
        elif ano_entrada == 2020:
            treinamento_dataset = Dados(dados_treinamento_2020, modelo_entrada)
        elif ano_entrada == 2022:
            treinamento_dataset = Dados(dados_treinamento_2022, modelo_entrada)
        treinamento_dataloader = DataLoader(treinamento_dataset, batch_size=32, shuffle=True)

        validacao_dataset = Dados(dados_validacao, modelo_entrada)
        validacao_dataloader = DataLoader(validacao_dataset, batch_size=32, shuffle=True)

        # Criando redes
        saida = 24
        rede = LSTM(n_entrada = n_entradas, n_primeira_camada = n_neuronios_1_lstm, n_segunda_camada = n_neuronios_2_lstm, n_saida = 24)
           
        # Definindo criterio e loss
        criterio = nn.MSELoss()

        if otm == 'Adam':
            otimizador = optim.Adam(rede.parameters(), lr = taxa_aprendizado)
        elif otm == 'Rprop':
            otimizador = optim.Rprop(rede.parameters(), lr = taxa_aprendizado)
        elif otm == 'RMSprop':
            otimizador = optim.RMSprop(rede.parameters(), lr = taxa_aprendizado)
        
        earlystop = EarlyStopping(n_earlystop)
        melhor_perda_validacao = np.inf

        treinamento_losses, validacao_losses = [], []

        for epoca in range(epocas):
            perda_treino, perda_validacao = treinamento(rede, treinamento_dataloader, validacao_dataloader, otimizador, criterio)
            
            treinamento_losses.append(perda_treino)
            validacao_losses.append(perda_validacao)

            if perda_validacao < melhor_perda_validacao:
                melhor_perda_validacao = perda_validacao

            # Early Stopping
            if earlystop(perda_validacao):
                break
        
        lista_melhor_loss.append(melhor_perda_validacao)

        
    return np.mean(lista_melhor_loss)

## Otimização

In [11]:
class AG:
    def __init__(self, limites, tamanho_populacao, geracoes, taxa_mutacao, funcao_custo):
        self.limites = limites
        self.numero_variaveis = len(self.limites)
        self.tamanho_populacao = tamanho_populacao
        self.geracoes = geracoes
        self.taxa_mutacao = taxa_mutacao
        self.funcao_custo = funcao_custo
        self.geracao_atual = 0

        intervalos = [range(a, b+1) for a, b in limites]
        possibilidades = [comb for comb in product(*intervalos)]
        self.dic_historico = {(possibilidade): [] for possibilidade in possibilidades}

        self.populacao = []

    def criar_individuo(self):
        return [round(random.uniform(a - 0.49, b + 0.49)) for (a, b) in self.limites]

    def criar_populacao(self):
        return [self.criar_individuo() for _ in range(self.tamanho_populacao)]

    def avaliar_fitness(self):
        custos = []
        for ind in self.populacao:
            chave = tuple(ind)
            if self.dic_historico[chave]:
                print(f'Indivíduo já avaliado.')
                custos.append(self.dic_historico[chave][0])
            else:
                fit = self.funcao_custo(ind)
                custos.append(fit)
                self.dic_historico[chave].append(fit)
            print(f'Indivíduo: {ind}. Fit: {custos[-1]}')
        self.fitness = [1 / (1 + c) for c in custos]

    def selecao_roleta(self):
        soma = sum(self.fitness)
        pick = random.uniform(0, soma)
        atual = 0
        for ind, fit in zip(self.populacao, self.fitness):
            atual += fit
            if atual > pick:
                return ind
        return self.populacao[-1]

    def cruzamento(self, pai1, pai2):
        n = len(pai1)
        indices_pai1 = random.sample(range(n), k=n//2)
        filho = [pai1[i] if i in indices_pai1 else pai2[i] for i in range(n)]
        return filho

    def mutacao(self, ind):
        for i in range(self.numero_variaveis):
            if random.random() < self.taxa_mutacao:
                a, b = self.limites[i]
                ind[i] = round(random.uniform(a - 0.49, b + 0.49))
        return ind

    def salvar(self, nome_arquivo='ag_estado.pkl'):
        with open(nome_arquivo, 'wb') as f:
            pickle.dump(self, f)
        print(f"Estado salvo em '{nome_arquivo}'.")

    @staticmethod
    def carregar(nome_arquivo='ag_estado.pkl'):
        with open(nome_arquivo, 'rb') as f:
            ag = pickle.load(f)
        print(f"Estado carregado de '{nome_arquivo}'.")
        return ag

    def algoritmo_genetico(self, continuar=False):
        if not continuar or not self.populacao:
            self.populacao = self.criar_populacao()

        for geracao in range(self.geracao_atual, self.geracoes):
            print(f'\nGeração {geracao + 1}')
            self.avaliar_fitness()

            # Elitismo de 2 melhores
            indices_melhores = sorted(range(len(self.fitness)), key=lambda i: self.fitness[i], reverse=True)[:2]
            elite = [self.populacao[i] for i in indices_melhores]
            nova_pop = elite[:]

            while len(nova_pop) < self.tamanho_populacao:
                pai1 = self.selecao_roleta()
                pai2 = self.selecao_roleta()
                filho = self.cruzamento(pai1, pai2)
                filho = self.mutacao(filho)
                nova_pop.append(filho)

            self.populacao = nova_pop
            print(f"Elite: {elite}")

            # Salvar estado atual
            self.salvar()
            self.geracao_atual += 1

### Rodada

In [12]:
busca_entrada = AG(
    limites = [(0, 11), (0, 4), (0, 4), (0, 2), (0, 2), (0, 2)], 
    tamanho_populacao = 60, 
    geracoes = 10, 
    taxa_mutacao = 0.03, 
    funcao_custo = treinar)

In [13]:
busca_entrada.algoritmo_genetico()


Geração 1
Indivíduo: [5, 3, 2, 2, 0, 0]. Fit: 0.004802154216061657
Indivíduo: [1, 0, 1, 2, 1, 1]. Fit: 0.033621896686963734
Indivíduo: [11, 1, 2, 2, 2, 1]. Fit: 0.008670173703770464
Indivíduo: [4, 3, 3, 1, 1, 0]. Fit: 0.014783502249823263
Indivíduo: [2, 2, 0, 2, 2, 0]. Fit: 0.016190594337725392
Indivíduo: [10, 0, 4, 2, 1, 0]. Fit: 0.005164243973558769
Indivíduo: [7, 3, 4, 2, 2, 1]. Fit: 0.0080361821455881
Indivíduo: [5, 0, 0, 1, 1, 1]. Fit: 0.009532044152729215
Indivíduo: [8, 1, 0, 0, 1, 1]. Fit: 0.026820364625503617
Indivíduo: [8, 0, 2, 0, 0, 0]. Fit: 0.013581956395258507
Indivíduo: [0, 1, 1, 0, 2, 0]. Fit: 0.01654363425526147
Indivíduo: [9, 1, 3, 2, 1, 1]. Fit: 0.010458999949817856
Indivíduo: [0, 4, 0, 2, 2, 0]. Fit: 0.01710236663542067
Indivíduo: [4, 2, 3, 2, 2, 0]. Fit: 0.017705516959540545
Indivíduo: [0, 4, 4, 1, 2, 0]. Fit: 0.016033887521674235
Indivíduo: [1, 1, 0, 2, 0, 2]. Fit: 0.015371543374688676
Indivíduo: [9, 1, 2, 0, 0, 1]. Fit: 0.010410830147642021
Indivíduo: [9, 0, 1, 1

In [14]:
modelo_entrada, n_neuronios_1_lstm, n_neuronios_2_lstm, ano_entrada, taxa_aprendizado, otm = busca_entrada.populacao[0]
lista_modelo_entrada = (
        [f"M1_{i}" for i in range(1, 2)] +
        [f"M2_{i}" for i in range(1, 5)] +
        [f"M3_{i}" for i in range(1, 6)] + 
        [f"M4_{i}" for i in range(1, 3)]
)
lista_modelo_entrada = (
        [f"M1_{i}" for i in range(1, 2)] +
        [f"M2_{i}" for i in range(1, 5)] +
        [f"M3_{i}" for i in range(1, 6)] + 
        [f"M4_{i}" for i in range(1, 3)]
)
lista_modelo_n_entradas = [48, 48+24, 48+3, 48+2, 48+2, 48+24+2, 48+24+2, 48+3+2, 48+3+2, 48+2+2, 48+24+2+2, 48+3+2+2]
n_entradas = lista_modelo_n_entradas[modelo_entrada]
modelo_entrada = lista_modelo_entrada[modelo_entrada]

lista_n_neuronios = [24, round(0.25*(2*n_entradas-24)+24), round(0.5*(2*n_entradas-24)+24), round(0.75*(2*n_entradas-24)+24), 2*n_entradas]
n_neuronios_1_lstm = lista_n_neuronios[n_neuronios_1_lstm]

lista_n_neuronios = [24, round(0.25*(n_neuronios_1_lstm-24)+24), round(0.5*(n_neuronios_1_lstm-24)+24), round(0.75*(n_neuronios_1_lstm-24)+24), n_neuronios_1_lstm]
n_neuronios_2_lstm = lista_n_neuronios[n_neuronios_2_lstm]

lista_ano_entrada = [2018, 2020, 2022]
ano_entrada = lista_ano_entrada[ano_entrada]

lista_taxa_aprendizado = [0.01, 0.001, 0.0001]
taxa_aprendizado = lista_taxa_aprendizado[taxa_aprendizado]

lista_otm = ['Adam', 'Rprop', 'RMSprop']
otm = lista_otm[otm]

print(f'O melhor modelo é: {modelo_entrada}, {n_neuronios_1_lstm}, {n_neuronios_2_lstm}, {ano_entrada}, {taxa_aprendizado}, {otm}')

O melhor modelo é: M3_1, 55, 47, 2018, 0.001, RMSprop
